## MD문서만들기

### 통신사 요금제 / 카드은행사 / 구독상품
- 커스텀 스플리터

In [22]:
import os
import re
from docx import Document

# --- 경로 설정 ---
SOURCE_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\카드_은행_0102"
MD_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\MD_개별파일"

# 처리 대상 리스트
card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

def finalize_unit(unit_dict, card_company):
    """
    수집된 정보를 정형화된 MD 텍스트 포맷으로 변환
    '카드사' 항목을 추가하여 출처를 명확히 함
    """
    ott_combined = ", ".join(unit_dict["OTT_혜택"]) if unit_dict["OTT_혜택"] else "정보 없음"
    annual_fee = unit_dict["연회비"] if unit_dict["연회비"] else "정보 없음"
    target = unit_dict["추천_대상"] if unit_dict["추천_대상"] else "정보 없음"
    
    # 요청하신 대로 '카드사' 라인을 추가함
    return (
        f"# {unit_dict['name']}\n"
        f"카드사 : {card_company}\n"
        f"연회비 : {annual_fee}\n"
        f"OTT 혜택 : {ott_combined}\n"
        f"추천 대상 : {target}"
    )

def generate_md_with_company():
    # 출력 폴더 생성
    os.makedirs(MD_OUTPUT_DIR, exist_ok=True)

    print("📂 카드사 정보가 포함된 MD 문서 생성 시작...")

    for card_name in card_list:
        source_path = os.path.join(SOURCE_DIR, f"{card_name}.docx")
        
        if not os.path.exists(source_path):
            print(f"⚠️ 파일을 찾을 수 없습니다 (건너뜀): {card_name}.docx")
            continue

        doc = Document(source_path)
        raw_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
        
        products_md = []
        current_unit = None

        for line in raw_lines:
            # 1. 불필요한 태그 제거
            line = re.sub(r'\\s*', '', line)
            
            # 2. 상품명 감지 (스플릿 지점)
            header_match = re.search(r'\[(.*?)\]\s*(.*)', line)
            if header_match:
                if current_unit:
                    # finalize_unit 호출 시 card_name(카드사)을 인자로 전달
                    products_md.append(finalize_unit(current_unit, card_name))
                
                p_name = header_match.group(2).strip()
                current_unit = {
                    "name": p_name,
                    "연회비": "",
                    "OTT_혜택": [],
                    "추천_대상": "",
                    "state": "normal"
                }
            
            elif current_unit:
                # 3. 데이터 매칭 로직
                if "연회비 :" in line:
                    current_unit["연회비"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif "OTT 혜택" in line:
                    current_unit["state"] = "ott"
                elif "추천 대상 :" in line:
                    current_unit["추천_대상"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif current_unit["state"] == "ott":
                    clean_benefit = re.sub(r'^[ln·-]\s+', '', line)
                    current_unit["OTT_혜택"].append(clean_benefit)

        # 마지막 상품 추가
        if current_unit:
            products_md.append(finalize_unit(current_unit, card_name))

        # 4. 파일 저장
        if products_md:
            md_file_path = os.path.join(MD_OUTPUT_DIR, f"{card_name}.md")
            full_md_text = "\n\n---\n\n".join(products_md)
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(full_md_text)
            
            print(f"✅ [{card_name}] MD 생성 완료 (카드사 정보 포함)")

if __name__ == "__main__":
    generate_md_with_company()

📂 카드사 정보가 포함된 MD 문서 생성 시작...
✅ [삼성카드] MD 생성 완료 (카드사 정보 포함)
✅ [KB국민카드] MD 생성 완료 (카드사 정보 포함)
✅ [롯데카드] MD 생성 완료 (카드사 정보 포함)
✅ [신한카드] MD 생성 완료 (카드사 정보 포함)
✅ [NH농협카드] MD 생성 완료 (카드사 정보 포함)
✅ [하나카드] MD 생성 완료 (카드사 정보 포함)
✅ [IBK기업은행] MD 생성 완료 (카드사 정보 포함)
✅ [우리카드] MD 생성 완료 (카드사 정보 포함)
✅ [BC바로카드] MD 생성 완료 (카드사 정보 포함)
✅ [현대카드] MD 생성 완료 (카드사 정보 포함)
✅ [iM뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [케이뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [BNK부산은행] MD 생성 완료 (카드사 정보 포함)
✅ [광주은행] MD 생성 완료 (카드사 정보 포함)
✅ [제주은행] MD 생성 완료 (카드사 정보 포함)
✅ [토스뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [우체국] MD 생성 완료 (카드사 정보 포함)
✅ [미래에셋증권] MD 생성 완료 (카드사 정보 포함)


## 임베딩 및 json 파일 만들기

In [8]:
import os
import re
import json
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\카드은행\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\카드은행\json_v2"

# 처리 대상 리스트
card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

# 청크 최대 글자 수
MAX_CHARS_PER_CHUNK = 1000

print("🧠 MD 기반 가변 청킹 프로세스 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def run_flexible_chunking():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for card_name in card_list:
        md_path = os.path.join(MD_DIR, f"{card_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ MD 파일을 찾을 수 없습니다: {card_name}.md")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 개별 상품 유닛 분리 
        units = [u.strip() for u in content.split('---') if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 글자 수(1000자)에 맞춘 병합 로직
        for unit in units:
            # 현재 누적 텍스트 + 다음 상품 유닛이 1000자를 넘으면 현재까지를 청크로 확정
            if len(temp_chunk_text) + len(unit) > MAX_CHARS_PER_CHUNK:
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 1000자 미만이면 계속 병합
                if temp_chunk_text:
                    temp_chunk_text += "\n\n---\n\n" + unit
                else:
                    temp_chunk_text = unit
        
        # 마지막 남은 텍스트 추가
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 청크별 임베딩 및 JSON 생성
        card_json_results = []
        for i, chunk_text in enumerate(final_chunks):
            # 첫 번째 상품명을 추출하여 source 필드 대표값으로 사용 
            header_match = re.search(r"#\s*(.+)", chunk_text)
            source_name = header_match.group(1).strip() if header_match else card_name

            dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
            
            card_json_results.append({
                "pk": f"{card_name}_CHUNK_{i+1:02d}",
                "source": card_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{card_name}.md",
                "page": 1,
                "row": i + 1
            })

        # 4. 카드사별 JSON 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{card_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(card_json_results, f, ensure_ascii=False, indent=4)
        
        print(f"✅ [{card_name}] 청킹 완료: {len(final_chunks)}개의 청크 생성")

if __name__ == "__main__":
    run_flexible_chunking()

🧠 MD 기반 가변 청킹 프로세스 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


✅ [삼성카드] 청킹 완료: 6개의 청크 생성
✅ [KB국민카드] 청킹 완료: 6개의 청크 생성
✅ [롯데카드] 청킹 완료: 4개의 청크 생성
✅ [신한카드] 청킹 완료: 10개의 청크 생성
✅ [NH농협카드] 청킹 완료: 3개의 청크 생성
✅ [하나카드] 청킹 완료: 3개의 청크 생성
✅ [IBK기업은행] 청킹 완료: 1개의 청크 생성
✅ [우리카드] 청킹 완료: 3개의 청크 생성
✅ [BC바로카드] 청킹 완료: 2개의 청크 생성
✅ [현대카드] 청킹 완료: 1개의 청크 생성
✅ [iM뱅크] 청킹 완료: 1개의 청크 생성
✅ [케이뱅크] 청킹 완료: 1개의 청크 생성
✅ [BNK부산은행] 청킹 완료: 1개의 청크 생성
✅ [광주은행] 청킹 완료: 1개의 청크 생성
✅ [제주은행] 청킹 완료: 1개의 청크 생성
✅ [토스뱅크] 청킹 완료: 1개의 청크 생성
✅ [우체국] 청킹 완료: 1개의 청크 생성
✅ [미래에셋증권] 청킹 완료: 1개의 청크 생성


## MD문서만들기
- 마크다운 스플리터 라이브러리 사용

In [ ]:
pip install langchain-text-splitters

## 글자수 900자로 청킹 (최적)

### 파일 로드 및 임베딩 모델 로드
- 카드,은행사 md파일 기반 임베딩
- 청크 최대 글자수 조정

In [ ]:
import os
import re
import json
import time
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\카드은행\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\카드_은행파일"

card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

# 청크 최대 글자 수 상향 (최소 청크 생성을 위해 1200자 설정)
MAX_CHARS_PER_CHUNK = 900

print(f"🧠 카드/은행 MD 기반 최소 청크(Max {MAX_CHARS_PER_CHUNK}자) 생성 프로세스 시작...")
# 임베딩 모델 로드
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
def run_flexible_chunking_optimized():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for card_name in card_list:
        md_path = os.path.join(MD_DIR, f"{card_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ MD 파일을 찾을 수 없습니다: {card_name}.md")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 상품 단위 분리 (다양한 형태의 구분선 대응)
        units = [u.strip() for u in re.split(r'\n-+\n|---', content) if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 그리디(Greedy) 병합 로직: 900자를 꽉 채울 때까지 병합
        separator = "\n\n---\n\n"
        for unit in units:
            # 병합 시 추가될 길이 계산 (구분선 포함)
            added_len = len(unit) + (len(separator) if temp_chunk_text else 0)

            if len(temp_chunk_text) + added_len > MAX_CHARS_PER_CHUNK:
                # 900자를 초과하게 되면 현재까지 뭉친 텍스트를 확정
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 900자 이내라면 구분선과 함께 병합
                if temp_chunk_text:
                    temp_chunk_text += separator + unit
                else:
                    temp_chunk_text = unit
        
        # 마지막 남은 텍스트 추가
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 청크별 임베딩 및 JSON 생성
        card_json_results = []
        print(f"📂 [{card_name}] 총 {len(final_chunks)}개 청크 생성 완료")
        
        for i, chunk_text in enumerate(final_chunks):
            # --- Rate Limit(0.1) 대응: 요청 사이 1.1초 대기 ---
            time.sleep(1.1)
            
            try:
                dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
                
                card_json_results.append({
                    "pk": f"{card_name}_CHUNK_{i+1:02d}",
                    "source": card_name,
                    "text": chunk_text,
                    "dense": dense_vec.tolist(),
                    "file_hash": f"{card_name}.md",
                    "page": {i}+1,
                    "row": 1,
                })
                print(f"   ㄴ Chunk {i+1:02d} 임베딩 완료 ({len(chunk_text)}자)")
            except Exception as e:
                print(f"   ❌ Chunk {i+1:02d} 오류 발생: {e}")
                time.sleep(10) # 차단 시 대기 시간 증가
                continue

        # 4. 카드사별 JSON 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{card_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(card_json_results, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    run_flexible_chunking_optimized()
    print("\n🚀 모든 카드/은행 파일의 최소 청킹 및 임베딩 작업이 완료되었습니다.")

# 구독상품

### 구독 상품 md 형성

In [ ]:
import os
import re
from docx import Document

# --- 경로 설정 ---
# 구독 문서가 있는 경로로 수정
SOURCE_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\구독_0102"
MD_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\구독상품\md"

# 구독 서비스 리스트 (폴더 내 파일명에 맞춰 추가)
subscription_list = [
    "KT구독", "T우주패스", "네이버멤버십", "배민클럽", "유독", "티빙"
]

def finalize_unit(unit_dict, service_category):
    ott_combined = ", ".join(unit_dict["OTT_혜택"]) if unit_dict["OTT_혜택"] else "정보 없음"
    price_info = unit_dict["이용료"] if unit_dict["이용료"] else "정보 없음"
    target = unit_dict["추천_대상"] if unit_dict["추천_대상"] else "정보 없음"
    
    return (
        f"# {unit_dict['name']}\n"
        f"서비스 : {service_category}\n"
        f"이용료 : {price_info}\n"
        f"OTT 혜택 : {ott_combined}\n"
        f"추천 대상 : {target}"
    )

def generate_subscription_md():
    os.makedirs(MD_OUTPUT_DIR, exist_ok=True)
    print("📂 구독 상품 정보가 포함된 MD 문서 생성 시작...")

    for service_name in subscription_list:
        source_path = os.path.join(SOURCE_DIR, f"{service_name}.docx")
        
        if not os.path.exists(source_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {service_name}.docx")
            continue

        doc = Document(source_path)
        raw_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
        
        products_md = []
        current_unit = None

        for line in raw_lines:
            # 1. 태그 및 불필요한 공백 제거
            line = re.sub(r'\\s*', '', line)
            
            # 2. 상품명 감지 ([KT 구독] 상품명 패턴)
            header_match = re.search(r'\[(.*?)\]\s*(.*)', line)
            if header_match:
                if current_unit:
                    products_md.append(finalize_unit(current_unit, service_name))
                
                p_name = header_match.group(2).strip()
                current_unit = {
                    "name": p_name,
                    "이용료": "",
                    "OTT_혜택": [],
                    "추천_대상": "",
                    "state": "normal"
                }
            
            elif current_unit:
                # 3. 데이터 매칭 (구독 전용 키워드: 이용료, OTT 혜택, 추천 대상)
                if "이용료 :" in line:
                    current_unit["이용료"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif "OTT 혜택" in line:
                    current_unit["state"] = "ott"
                elif "추천 대상 :" in line:
                    current_unit["추천_대상"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif current_unit["state"] == "ott":
                    # 상세 혜택 수집 (요금제 정보 등)
                    clean_benefit = re.sub(r'^[ln·-]\s+', '', line)
                    current_unit["OTT_혜택"].append(clean_benefit)

        if current_unit:
            products_md.append(finalize_unit(current_unit, service_name))

        if products_md:
            md_file_path = os.path.join(MD_OUTPUT_DIR, f"{service_name}.md")
            full_md_text = "\n\n---\n\n".join(products_md)
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(full_md_text)
            
            print(f"✅ [{service_name}] MD 생성 완료")

if __name__ == "__main__":
    generate_subscription_md()

📂 구독 상품 정보가 포함된 MD 문서 생성 시작...
✅ [KT구독] MD 생성 완료
✅ [T우주패스] MD 생성 완료
✅ [네이버멤버십] MD 생성 완료
✅ [배민클럽] MD 생성 완료
✅ [유독] MD 생성 완료
✅ [티빙] MD 생성 완료


### 구독상풍 청킹

In [ ]:
import os
import re
import json
import time
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
# 1. 구독 상품 경로로 정확히 설정
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\구독상품\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\구독상품"

subscription_list = ["KT구독", "T우주패스", "네이버멤버십", "배민클럽", "유독", "티빙"]

# 청크 최대 글자 수 (1200자)
MAX_CHARS_PER_CHUNK = 900

print(f"🧠 {MAX_CHARS_PER_CHUNK}자 기준 최소 청크 생성 프로세스 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def run_minimized_chunking():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for service_name in subscription_list:
        md_path = os.path.join(MD_DIR, f"{service_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {md_path}")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 상품 단위 분리 (정규식으로 다양한 대시 형태 대응)
        units = [u.strip() for u in re.split(r'\n-+\n|---', content) if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 최소 청크를 위한 타이트한 병합 로직
        for unit in units:
            # 구분선 길이를 포함하여 계산 (\n\n---\n\n = 약 7자)
            separator = "\n\n---\n\n"
            current_len = len(temp_chunk_text)
            next_unit_len = len(unit)
            
            # 이미 텍스트가 있다면 구분선 길이까지 포함해서 체크
            added_len = next_unit_len + (len(separator) if current_len > 0 else 0)

            if current_len + added_len > MAX_CHARS_PER_CHUNK:
                # 다음 유닛을 넣었을 때 1200자를 넘으면, 지금까지의 뭉치를 저장
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 1200자 이내라면 계속 붙임
                if temp_chunk_text:
                    temp_chunk_text += separator + unit
                else:
                    temp_chunk_text = unit
        
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 결과 저장 및 확인 로그
        print(f"📊 [{service_name}] 총 텍스트 약 {len(content)}자 -> {len(final_chunks)}개 청크로 압축됨")
        
        service_results = []
        for i, chunk_text in enumerate(final_chunks):
            # 청크별 실제 길이 출력 (확인용)
            print(f"   ㄴ Chunk {i+1:02d}: {len(chunk_text)}자")
            
            time.sleep(1.0) # Rate Limit 대응
            dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
            
            service_results.append({
                "pk": f"{service_name}_CHUNK_{i+1:02d}",
                "source": service_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{service_name}.md",
                "page": 1,
                "row": i + 1
            })

        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{service_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(service_results, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    run_minimized_chunking()

🧠 900자 기준 최소 청크 생성 프로세스 시작...
📊 [KT구독] 총 텍스트 약 1845자 -> 3개 청크로 압축됨
   ㄴ Chunk 01: 803자
   ㄴ Chunk 02: 894자
   ㄴ Chunk 03: 132자
📊 [T우주패스] 총 텍스트 약 2588자 -> 4개 청크로 압축됨
   ㄴ Chunk 01: 817자
   ㄴ Chunk 02: 795자
   ㄴ Chunk 03: 811자
   ㄴ Chunk 04: 144자
📊 [네이버멤버십] 총 텍스트 약 166자 -> 1개 청크로 압축됨
   ㄴ Chunk 01: 166자
📊 [배민클럽] 총 텍스트 약 255자 -> 1개 청크로 압축됨
   ㄴ Chunk 01: 255자
📊 [유독] 총 텍스트 약 2044자 -> 3개 청크로 압축됨
   ㄴ Chunk 01: 873자
   ㄴ Chunk 02: 859자
   ㄴ Chunk 03: 298자
📊 [티빙] 총 텍스트 약 299자 -> 1개 청크로 압축됨
   ㄴ Chunk 01: 299자


### 텍스트 스플리터 라이브러리 사용
- 구독 상품(C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\구독상품\md)
    - 한 청크 내 글자 수 900
    - 한 청크당 7개의 구독상품 정보가 담김

    
- 카드 및 은행 상품
    - 한 청크 내 글자 수 900
    - 한 청크당 4~5개의 카드은행 정보가 담김

- 요금제
    - 한 청크 내 글자 수 1000자
    - 한 청크당 6개의 요금제 정보가 담김

## 요금제

In [ ]:
transformers.utils.move_cache()

In [1]:
import os
import re
import json
import time
import pandas as pd
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer

# --- 1. 경로 및 설정 ---
# 이전에 MD를 저장했던 경로로 설정하세요.
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\json_v4"

# 통신사 리스트
comp_list = ['SKT_v2', 'KT_v2', 'LGU+_v2']

# 청크 최대 글자 수 (이전 베이스라인과 동일하게 1000자 설정)
MAX_CHARS_PER_CHUNK = 800

print(f"🧠 [{', '.join(comp_list)}] 기반 텍스트 스플리터 청킹 프로세스 시작...")

# 2. 임베딩 모델 로드 (nomic-ai)
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 3. 마크다운 헤더 스플리터 설정
# # (H1) 기준으로 요금제를 인식합니다.
headers_to_split_on = [("#", "ProductName")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

def run_telecom_embedding():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for comp_name in comp_list:
        md_path = os.path.join(MD_DIR, f"{comp_name}.md")
        if not os.path.exists(md_path): continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # [STEP 1] 헤더 기반 분할
        docs = markdown_splitter.split_text(content)
        
        # [STEP 2] 그리디(Greedy) 병합 로직 수정
        final_chunks = []
        temp_text = ""
        separator = "\n\n---\n\n"

        for doc in docs:
            p_name = doc.metadata.get("ProductName", "알 수 없는 요금제")
            clean_content = re.sub(r'\n-+\n|---', '', doc.page_content).strip()
            
            # 다시 깨끗하게 유닛 구성
            chunk_unit = f"# {p_name}\n{clean_content}"
            
            added_len = len(chunk_unit) + (len(separator) if temp_text else 0)

            if len(temp_text) + added_len > MAX_CHARS_PER_CHUNK:
                if temp_text:
                    final_chunks.append(temp_text.strip())
                temp_text = chunk_unit
            else:
                if temp_text:
                    temp_text += separator + chunk_unit
                else:
                    temp_text = chunk_unit
        
        if temp_text:
            final_chunks.append(temp_text.strip())

        # [STEP 3] 임베딩 및 JSON 결과 생성
        print(f"📊 [{comp_name}] {len(docs)}개 요금제 -> {len(final_chunks)}개 청크로 최적화됨")
        
        comp_results = []
        for i, chunk_text in enumerate(final_chunks):
            # Rate Limit 대응 (1.1초 대기)
            time.sleep(1.1)
            
            # Nomic 모델 권장 접두사 추가 (검색 품질 향상)
            input_text = f"search_document: {chunk_text}"
            dense_vec = model.encode(input_text, normalize_embeddings=True)
            
            comp_results.append({
                "pk": f"{comp_name}_CHUNK_{i+1:02d}",
                "source": comp_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{comp_name}.md",
                "page": 1,
                "row": i + 1
            })
            print(f"   ㄴ Chunk {i+1:02d}: {len(chunk_text)}자 임베딩 완료")

        # 결과 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{comp_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(comp_results, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    run_telecom_embedding()
    print("\n🚀 모든 통신사 데이터의 라이브러리 기반 청킹 및 임베딩이 완료되었습니다.")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

🧠 [SKT_v2, KT_v2, LGU+_v2] 기반 텍스트 스플리터 청킹 프로세스 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


📊 [SKT_v2] 138개 요금제 -> 54개 청크로 최적화됨
   ㄴ Chunk 01: 756자 임베딩 완료
   ㄴ Chunk 02: 541자 임베딩 완료
   ㄴ Chunk 03: 563자 임베딩 완료
   ㄴ Chunk 04: 539자 임베딩 완료
   ㄴ Chunk 05: 792자 임베딩 완료
   ㄴ Chunk 06: 573자 임베딩 완료
   ㄴ Chunk 07: 554자 임베딩 완료
   ㄴ Chunk 08: 564자 임베딩 완료
   ㄴ Chunk 09: 794자 임베딩 완료
   ㄴ Chunk 10: 527자 임베딩 완료
   ㄴ Chunk 11: 537자 임베딩 완료
   ㄴ Chunk 12: 557자 임베딩 완료
   ㄴ Chunk 13: 747자 임베딩 완료
   ㄴ Chunk 14: 794자 임베딩 완료
   ㄴ Chunk 15: 543자 임베딩 완료
   ㄴ Chunk 16: 779자 임베딩 완료
   ㄴ Chunk 17: 572자 임베딩 완료
   ㄴ Chunk 18: 800자 임베딩 완료
   ㄴ Chunk 19: 571자 임베딩 완료
   ㄴ Chunk 20: 795자 임베딩 완료
   ㄴ Chunk 21: 786자 임베딩 완료
   ㄴ Chunk 22: 763자 임베딩 완료
   ㄴ Chunk 23: 796자 임베딩 완료
   ㄴ Chunk 24: 782자 임베딩 완료
   ㄴ Chunk 25: 692자 임베딩 완료
   ㄴ Chunk 26: 533자 임베딩 완료
   ㄴ Chunk 27: 556자 임베딩 완료
   ㄴ Chunk 28: 540자 임베딩 완료
   ㄴ Chunk 29: 530자 임베딩 완료
   ㄴ Chunk 30: 557자 임베딩 완료
   ㄴ Chunk 31: 537자 임베딩 완료
   ㄴ Chunk 32: 562자 임베딩 완료
   ㄴ Chunk 33: 775자 임베딩 완료
   ㄴ Chunk 34: 782자 임베딩 완료
   ㄴ Chunk 35: 759자 임베딩 완료
   ㄴ Chunk 36: 760자

In [10]:
import os
import re
import json
import time
import pandas as pd
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer

# --- 1. 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\json_v3"

comp_list = ['SKT_v2', 'KT_v2', 'LGU+_v2']
MAX_CHARS_PER_CHUNK = 800

print(f"🧠 [{', '.join(comp_list)}] 기반 텍스트 스플리터 청킹 프로세스 시작...")

# 2. 임베딩 모델 로드
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 3. 마크다운 헤더 스플리터 설정
headers_to_split_on = [("#", "ProductName")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

def run_telecom_embedding():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for comp_name in comp_list:
        md_path = os.path.join(MD_DIR, f"{comp_name}.md")
        if not os.path.exists(md_path): 
            print(f"⚠️ {comp_name}.md 파일을 찾을 수 없어 건너뜁니다.")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # [STEP 1] 헤더 기반 1차 분할
        docs = markdown_splitter.split_text(content)
        print(docs)
        # [STEP 2] 그리디(Greedy) 병합 로직
        final_chunks = []
        temp_text = ""
        separator = "\n\n---\n\n"

        for doc in docs:
            p_name = doc.metadata.get("ProductName", "알 수 없는 요금제")
            clean_content = re.sub(r'\n-+\n|---', '', doc.page_content).strip()
            chunk_unit = f"# {p_name}\n{clean_content}"
            print(doc)
            added_len = len(chunk_unit) + (len(separator) if temp_text else 0)

            if len(temp_text) + added_len > MAX_CHARS_PER_CHUNK:
                if temp_text:
                    final_chunks.append(temp_text.strip())
                temp_text = chunk_unit
            else:
                if temp_text:
                    temp_text += separator + chunk_unit
                else:
                    temp_text = chunk_unit
        
        if temp_text:
            final_chunks.append(temp_text.strip())

        # --- [DEBUG] 스플리터 결과 구조 확인 섹션 ---
        print(f"\n" + "="*50)
        print(f"🔍 [{comp_name}] 데이터 구조 분석 보고서")
        print(f"  - 원본 요금제 개수: {len(docs)}개")
        print(f"  - 병합 후 최종 청크: {len(final_chunks)}개")
        
        # 간단한 데이터프레임 생성하여 구조 시각화
        debug_list = []
        for i, chunk in enumerate(final_chunks):
            # 해당 청크에 포함된 요금제 이름들 추출 (# 제목 추출)
            included_products = re.findall(r'^#\s+(.+)$', chunk, re.MULTILINE)
            debug_list.append({
                "ID": i+1,
                "Length": len(chunk),
                "Product_Count": len(included_products),
                "Products": ", ".join(included_products)
            })
        
        df_debug = pd.DataFrame(debug_list)
        print("\n[청크별 요약 테이블]")
        print(df_debug.to_string(index=False))
        
        print(f"\n[첫 번째 청크 실제 구조 샘플]")
        print("-" * 30)
        print(final_chunks[0][:500] + ("..." if len(final_chunks[0]) > 500 else ""))
        print("-" * 30)
        print("="*50 + "\n")
        # ------------------------------------------

        # [STEP 3] 임베딩 및 JSON 결과 생성
        comp_results = []
        for i, chunk_text in enumerate(final_chunks):
            # Rate Limit 대응 (임베딩 API 사용 시 필요)
            time.sleep(1.1)
            
            input_text = f"search_document: {chunk_text}"
            dense_vec = model.encode(input_text, normalize_embeddings=True)
            
            comp_results.append({
                "pk": f"{comp_name}_CHUNK_{i+1:02d}",
                "source": comp_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{comp_name}.md",
                "page": 1,
                "row": i + 1
            })
            print(f" ✅ Chunk {i+1:02d} 임베딩 완료 ({len(chunk_text)}자)")

        # 결과 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{comp_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(comp_results, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    run_telecom_embedding()
    print("\n🚀 모든 통신사 데이터의 구조 확인 및 임베딩이 완료되었습니다.")

ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?

In [1]:
import os
import re
import json
import time
import pandas as pd
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer

# --- 1. 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\json_v2"

comp_list = ['SKT']
MAX_CHARS_PER_CHUNK = 1000

print(f"🧠 [{', '.join(comp_list)}] 기반 텍스트 스플리터 청킹 프로세스 시작...")

# 2. 임베딩 모델 로드
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 3. 마크다운 헤더 스플리터 설정
headers_to_split_on = [("#", "ProductName")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

🧠 [SKT] 기반 텍스트 스플리터 청킹 프로세스 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


In [2]:
def run_telecom_embedding():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for comp_name in comp_list:
        md_path = os.path.join(MD_DIR, f"{comp_name}.md")
        if not os.path.exists(md_path): 
            print(f"⚠️ {comp_name}.md 파일을 찾을 수 없어 건너뜁니다.")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # [STEP 1] 헤더 기반 1차 분할
        docs = markdown_splitter.split_text(content)
        # [STEP 2] 그리디(Greedy) 병합 로직
        final_chunks = []
        temp_text = ""
        separator = "\n\n---\n\n"

        for doc in docs:
            p_name = doc.metadata.get("ProductName", "알 수 없는 요금제")
            clean_content = re.sub(r'\n-+\n|---', '', doc.page_content).strip()
            chunk_unit = f"# {p_name}\n{clean_content}"
            print(doc)
            added_len = len(chunk_unit) + (len(separator) if temp_text else 0)

            if len(temp_text) + added_len > MAX_CHARS_PER_CHUNK:
                if temp_text:
                    final_chunks.append(temp_text.strip())
                temp_text = chunk_unit
            else:
                if temp_text:
                    temp_text += separator + chunk_unit
                else:
                    temp_text = chunk_unit

In [3]:
run_telecom_embedding()

page_content='통신사 : SKT
이용료 : 26,000원/월
OTT 혜택 : 정보 없음
주요 특징 : 5G 스마트폰을 처음 사용하는 청소년을 위한 SKT의 합리적인 실속형 플랜
통신망 : 5G
데이터 제공량 : 3GB (속도제한)
음성 제공량 : 제한없음
추천 대상 : 집이나 학교에서 와이파이 환경 사용이 주를 이루며, 5G 서비스를 저렴하게 이용하려는 초등학생이나 중학생 자녀에게 추천  
---' metadata={'ProductName': '요금제명 : 5G ZEM플랜 베스트'}
page_content='통신사 : SKT
이용료 : 36,000원/월
OTT 혜택 : 정보 없음
주요 특징 : 5G 시대를 처음 경험하는 어린이를 위해 설계된 합리적인 가격의 필수형 요금제
통신망 : 5G
데이터 제공량 : 6GB (속도제한)
음성 제공량 : 제한없음
추천 대상 : 통화량이 많고 와이파이 환경에서 주로 인터넷을 사용하며, 과도한 데이터 사용을 막고자 하는 초등학생 및 저학년 청소년에게 적합  
---' metadata={'ProductName': '요금제명 : 5G ZEM플랜 퍼펙트'}
page_content='통신사 : SKT
이용료 : 26,000원/월
OTT 혜택 : 정보 없음
주요 특징 : 2만원대 초저가로 통화는 무제한, 데이터는 실속 있게 제공하는 통화 중심형 요금제
통신망 : LTE
데이터 제공량 : 2GB (속도제한)
음성 제공량 : 제한없음
추천 대상 : 데이터를 거의 사용하지 않으며, 저렴한 월 요금과 무제한 통화 혜택이 가장 중요한 실버 세대 또는 초등학생 자녀에게 적합  
---' metadata={'ProductName': '요금제명 : ZEM플랜 베스트'}
page_content='통신사 : SKT
이용료 : 109,000원/월
OTT 혜택 : 정보 없음
주요 특징 : 데이터 무제한은 기본, 커피/영화/로밍 할인을 청년들에게 쏟아붓는 프리미엄 라이프스타일 요금제
통신망 : 5G
데이터 제공량 : 무제한
음성 제공량 : 제한없음
